In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import seaborn as sns
import requests

/Users/jordankesner/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [18]:
response = requests.get("https://api.openf1.org/v1/sessions?year=2023")
sessions = pd.DataFrame(response.json())
race_sessions = sessions[sessions["session_type"] == "Race"]
race_sessions[["meeting_key", "session_key", "session_name"]].head()

,meeting_key,session_key,session_name
7,1141,7953,Race
12,1142,7779,Race
17,1143,7787,Race
21,1207,9069,Sprint
22,1207,9070,Race


In [19]:
sessions["session_type"].value_counts()

session_type
Practice      57
Qualifying    28
Race          28
Name: count, dtype: int64

In [ ]:
def getRaceKeys(year):
    """
    Get the session keys for all races in a given year.
    """
    url = f"https://api.openf1.org/v1/sessions?year={year}"
    response = requests.get(url)
    sessions = pd.DataFrame(response.json())
    race_sessions = sessions[sessions["session_type"] == "Race"]
    return race_sessions["session_key"].tolist()

In [24]:
keys = getRaceKeys("2023")

In [26]:
len(keys)

28

In [48]:
def getFinalPositions(session_key):
    """
    Get final race positions (1 per driver) using latest timestamp from /position.
    """
    url = f"https://api.openf1.org/v1/position?session_key={session_key}"
    response = requests.get(url)
    
    if response.status_code != 200:
        raise Exception(f"Failed to fetch positions for session_key={session_key}")
    
    df = pd.DataFrame(response.json())
    if df.empty:
        print(f"No data for session {session_key}")
        return pd.DataFrame()

    df["date"] = pd.to_datetime(df["date"])

    # Get latest position for each driver
    df = df.sort_values("date")
    last_positions = df.groupby("driver_number").tail(1)  # ✅ Get only last row per driver

    return last_positions[["driver_number", "position", "date"]]


In [60]:
race = getFinalPositions(7787)

In [61]:
race

,driver_number,position,date
55,16,20,2023-04-02 05:04:03.949000+00:00
83,14,3,2023-04-02 05:15:10.269000+00:00
124,23,19,2023-04-02 05:15:40.785000+00:00
161,1,1,2023-04-02 05:41:03.455000+00:00
162,44,2,2023-04-02 05:41:03.455000+00:00
209,63,18,2023-04-02 05:50:08.917000+00:00
254,20,17,2023-04-02 06:39:53.148000+00:00
258,2,16,2023-04-02 07:00:04.900000+00:00
259,21,15,2023-04-02 07:00:04.900000+00:00
278,10,13,2023-04-02 07:33:29.049000+00:00


In [62]:
drivers = pd.DataFrame(requests.get("https://api.openf1.org/v1/drivers").json())

In [63]:
drivers.head()

,meeting_key,session_key,driver_number,broadcast_name,full_name,name_acronym,team_name,team_colour,first_name,last_name,headshot_url,country_code
0,1140,7763,1,M VERSTAPPEN,Max VERSTAPPEN,VER,Red Bull Racing,3671C6,Max,Verstappen,https://www.formula1.com/content/dam/fom-websi...,NED
1,1140,7763,2,L SARGEANT,Logan SARGEANT,SAR,Williams,37BEDD,Logan,Sargeant,https://www.formula1.com/content/dam/fom-websi...,USA
2,1140,7763,4,L NORRIS,Lando NORRIS,NOR,McLaren,F58020,Lando,Norris,https://www.formula1.com/content/dam/fom-websi...,GBR
3,1140,7763,10,P GASLY,Pierre GASLY,GAS,Alpine,2293D1,Pierre,Gasly,https://www.formula1.com/content/dam/fom-websi...,FRA
4,1140,7763,11,S PEREZ,Sergio PEREZ,PER,Red Bull Racing,3671C6,Sergio,Perez,https://www.formula1.com/content/dam/fom-websi...,MEX


In [64]:
drivers = drivers.drop_duplicates(subset="driver_number", keep="last")


In [65]:
race = race.merge(drivers[["driver_number","full_name","team_name"]], on="driver_number", how="left")

In [69]:
race.sort_values("position", inplace=True)
race

,driver_number,position,date,full_name,team_name
3,1,1,2023-04-02 05:41:03.455000+00:00,Max VERSTAPPEN,Red Bull Racing
4,44,2,2023-04-02 05:41:03.455000+00:00,Lewis HAMILTON,Ferrari
1,14,3,2023-04-02 05:15:10.269000+00:00,Fernando ALONSO,Aston Martin
14,18,4,2023-04-02 07:36:02.579000+00:00,Lance STROLL,Aston Martin
12,11,5,2023-04-02 07:36:02.579000+00:00,Sergio PEREZ,Red Bull Racing
13,4,6,2023-04-02 07:36:02.579000+00:00,Lando NORRIS,McLaren
11,27,7,2023-04-02 07:36:02.579000+00:00,Nico HULKENBERG,Kick Sauber
15,81,8,2023-04-02 07:36:02.735000+00:00,Oscar PIASTRI,McLaren
16,24,9,2023-04-02 07:36:03.079000+00:00,ZHOU Guanyu,Kick Sauber
17,22,10,2023-04-02 07:36:03.407000+00:00,Yuki TSUNODA,Red Bull Racing
